In [131]:
# Importing all the necessary packages
import requests
import json
import pandas as pd
from pandas import json_normalize

In [152]:
# Function to extract data from The Graph using api

def extract(skip_val):
    url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2"
    query = """query swaps{
      swaps(first: 1000,skip:"""+str(skip_val)+""",orderBy: timestamp, orderDirection: desc, where:{pair: "0xd3d2e2692501a5c9ca623199d38826e513033a17"}) {
        pair{
        id
        }
        timestamp
        amount0In
        amount1In
        amount0Out
        amount1Out
        pair {
          token0 {
            symbol
            totalLiquidity
          }
          token1 {
            symbol
            totalLiquidity
          }

        }
        amountUSD

      }
    }

    """
    
    read = requests.post(url, json={'query':query})
    json_data = json.loads(read.text)
    
    # Normalising the json data, such that it can be converted into a pandas data frame.
    json_data = json_data['data']
    json_data = json_data['swaps']
    df = json_normalize(json_data)
    return df

In [165]:
# Extracting the first 1000 records
df = extract(0)

# Extracting another 4000 records, a total of 5000 records
for i in range(1000,5000,1000):
    df = df.append(extract(i))

In [166]:
df.head(5)

,amount0In,amount0Out,amount1In,amount1Out,amountUSD,timestamp,pair.id,pair.token0.symbol,pair.token0.totalLiquidity,pair.token1.symbol,pair.token1.totalLiquidity
0,0,115.82195658391726228,1.012235403194200641,0,1931.120438260540614988148851667291,1624385907,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
1,0,1420.520778718993600793,12.42437377678812313,0,23657.0666517494874405829049185857,1624385784,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
2,1670.460789158621855977,0,0,14.521398844699505991,27649.98111916933588601798285662503,1624385784,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
3,1450.394072773710361932,0,0,12.609818873267061482,23998.37706594808809849389728259066,1624385784,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
4,0,449.256876000463203074,3.931258869321400678,0,7480.742391612794467421822598719475,1624385751,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809


In [167]:
# Reseting the index
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df.tail(5)

,amount0In,amount0Out,amount1In,amount1Out,amountUSD,timestamp,pair.id,pair.token0.symbol,pair.token0.totalLiquidity,pair.token1.symbol,pair.token1.totalLiquidity
4995,0,529.931342411848193832,5.105945261833397248,0,13298.74255273496426023388721273017,1623279079,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
4996,0.360894275112212837,0,0,0.003455644545584,9.004866085906828142528798281273083,1623279041,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
4997,36.609399965775007261,0,0,0.350577100571877374,913.5138536828944285381899525818718,1623279015,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
4998,95.015978666026611924,0,0,0.909836442255675626,2370.832917958496960558306380828813,1623279015,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809
4999,39.611433520143703485,0,0,0.379337299493463807,988.4568234015639480838637278973852,1623278994,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809


In [177]:
# Converting timestamp into Date Time format
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [186]:
# Checking the range of dates
start_date = df.iloc[0]['timestamp']
end_date = df.iloc[-1]['timestamp']
print("The Data ranges between the date "+str(end_date)+" to "+str(start_date))

The Data ranges between the date 2021-06-09 22:49:54 to 2021-06-22 18:18:27


In [194]:
# Creating a column for side(Buy/Sell)
    # amount0In and amount0Out -> represents UNI
    # amount1In and amount1Out -> represents WETH
        # if amount0In is 0 and amount0Out has some value, it means that UNI is sent out of the wallet, i.e. Sell
        # if amount0Out is 0 and amount0In has some value. it means that UNI is recived to the wallet, i.e. Buy  

df['side'] = ["Sell" if float(x)==0 else "Buy" for x in df['amount0In']]

In [197]:
df.head(2)

,amount0In,amount0Out,amount1In,amount1Out,amountUSD,timestamp,pair.id,pair.token0.symbol,pair.token0.totalLiquidity,pair.token1.symbol,pair.token1.totalLiquidity,side
0,0,115.82195658391726228,1.012235403194200641,0,1931.120438260540614988148851667291,2021-06-22 18:18:27,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809,Sell
1,0,1420.520778718993600793,12.42437377678812313,0,23657.0666517494874405829049185857,2021-06-22 18:16:24,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809,Sell


In [220]:
# converting
    # amount0In
    # amount1In
    # pair.token0.symbol
        # into "Base curency quantity traded" format

# Eg
    # amount0In/amount0Out UNI
    # 115.82195 UNI    -> rounded off to 5 decimals  -> it is not appropriate to round off a digital currency
    
b_c = df['amount0In']+df['amount0Out']
b_c = [round(float(i),5) for i in b_c]
b_c = [(str(i)+" "+str(df['pair.token0.symbol'][0])) for i in b_c]
df["baseCurrency"] = b_c

q_o = df['amount1In']+df['amount1Out']
q_o = [round(float(i),5) for i in q_o]
q_o = [(str(i)+" "+str(df['pair.token1.symbol'][0])) for i in q_o]
df["quoteCurrency"] = q_o

df.head(2)

,amount0In,amount0Out,amount1In,amount1Out,amountUSD,timestamp,pair.id,pair.token0.symbol,pair.token0.totalLiquidity,pair.token1.symbol,pair.token1.totalLiquidity,side,baseCurrency,quoteCurrency
0,0,115.82195658391726228,1.012235403194200641,0,1931.120438260540614988148851667291,2021-06-22 18:18:27,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809,Sell,115.82196 UNI,1.01224 WETH
1,0,1420.520778718993600793,12.42437377678812313,0,23657.0666517494874405829049185857,2021-06-22 18:16:24,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2354994.06913136134888896,WETH,743976.706441449341688809,Sell,1420.52078 UNI,12.42437 WETH


In [224]:
# Creating a new column for total liquidity
df['pair.token1.totalLiquidity'] = pd.to_numeric(df['pair.token1.totalLiquidity'])
df['pair.token0.totalLiquidity'] = pd.to_numeric(df['pair.token0.totalLiquidity'])
df['totalLiquidity'] = round(df['pair.token1.totalLiquidity']+df['pair.token0.totalLiquidity'],5)
df.head(2)

,amount0In,amount0Out,amount1In,amount1Out,amountUSD,timestamp,pair.id,pair.token0.symbol,pair.token0.totalLiquidity,pair.token1.symbol,pair.token1.totalLiquidity,side,baseCurrency,quoteCurrency,totalLiquidity
0,0,115.82195658391726228,1.012235403194200641,0,1931.120438260540614988148851667291,2021-06-22 18:18:27,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2.354994e+06,WETH,743976.706441,Sell,115.82196 UNI,1.01224 WETH,3.098971e+06
1,0,1420.520778718993600793,12.42437377678812313,0,23657.0666517494874405829049185857,2021-06-22 18:16:24,0xd3d2e2692501a5c9ca623199d38826e513033a17,UNI,2.354994e+06,WETH,743976.706441,Sell,1420.52078 UNI,12.42437 WETH,3.098971e+06


In [226]:
# Removing excess columns
df.drop(['amount0In','amount0Out','amount1In','amount1Out','pair.token0.symbol','pair.token1.symbol','pair.token1.totalLiquidity','pair.token0.totalLiquidity'],axis=1,inplace=True)
df.head(2)

,amountUSD,timestamp,pair.id,side,baseCurrency,quoteCurrency,totalLiquidity
0,1931.120438260540614988148851667291,2021-06-22 18:18:27,0xd3d2e2692501a5c9ca623199d38826e513033a17,Sell,115.82196 UNI,1.01224 WETH,3.098971e+06
1,23657.0666517494874405829049185857,2021-06-22 18:16:24,0xd3d2e2692501a5c9ca623199d38826e513033a17,Sell,1420.52078 UNI,12.42437 WETH,3.098971e+06


In [227]:
# Re-naming columns
df.rename({'amountUSD': 'volumeUSD', 'pair.id': 'poolId'}, axis=1, inplace=True)
df.head(2)

,volumeUSD,timestamp,poolId,side,baseCurrency,quoteCurrency,totalLiquidity
0,1931.120438260540614988148851667291,2021-06-22 18:18:27,0xd3d2e2692501a5c9ca623199d38826e513033a17,Sell,115.82196 UNI,1.01224 WETH,3.098971e+06
1,23657.0666517494874405829049185857,2021-06-22 18:16:24,0xd3d2e2692501a5c9ca623199d38826e513033a17,Sell,1420.52078 UNI,12.42437 WETH,3.098971e+06


In [228]:
# Re-arranging the columns
df = df[['timestamp', 'side', 'baseCurrency', 'quoteCurrency', 'volumeUSD','totalLiquidity','poolId']]
df.head()

,timestamp,side,baseCurrency,quoteCurrency,volumeUSD,totalLiquidity,poolId
0,2021-06-22 18:18:27,Sell,115.82196 UNI,1.01224 WETH,1931.120438260540614988148851667291,3.098971e+06,0xd3d2e2692501a5c9ca623199d38826e513033a17
1,2021-06-22 18:16:24,Sell,1420.52078 UNI,12.42437 WETH,23657.0666517494874405829049185857,3.098971e+06,0xd3d2e2692501a5c9ca623199d38826e513033a17
2,2021-06-22 18:16:24,Buy,1670.46079 UNI,14.5214 WETH,27649.98111916933588601798285662503,3.098971e+06,0xd3d2e2692501a5c9ca623199d38826e513033a17
3,2021-06-22 18:16:24,Buy,1450.39407 UNI,12.60982 WETH,23998.37706594808809849389728259066,3.098971e+06,0xd3d2e2692501a5c9ca623199d38826e513033a17
4,2021-06-22 18:15:51,Sell,449.25688 UNI,3.93126 WETH,7480.742391612794467421822598719475,3.098971e+06,0xd3d2e2692501a5c9ca623199d38826e513033a17
